In [1]:
# 需要结合对模型的读取以及不同ratio数据的处理加入GMM
# 实现路径：1. 先读取model，做到可以计算结果
#2.再根据不同的给定的label情况，拟合GMM计算分类准确度， 输出图表
import sys
sys.path.append('..')
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
import code_package.tools_func as tool
import code_package.model_package as mcr2_model
DEVICE = 'cuda'
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 重新读取数据
def cal_test_performance(data_set, mcr2, set_name='test'):
    t_model_sensor_data = data_set[set_name][0].cuda()
    t_model_con_data = data_set[set_name][1].cuda()
    partial_label_input = data_set[set_name][2].cuda()
    true_label = data_set[set_name][3]
    mcr2.cuda()
    with torch.no_grad():
        res = mcr2(t_model_sensor_data, t_model_con_data, partial_label_input)
    loss = mcr2.loss_function(res,t_model_sensor_data)
    pred_label = torch.argmax(res['prob_oo'], dim=1).cpu().numpy()
    res = {}
    res['loss'] = loss['loss'].item()
    res['reconstruction_loss'] = loss['recons_loss'].item()
    res['kl_loss'] = loss['kl_loss'].item()
    res['acc'] = tool.cal_acc(pred_label, true_label.reshape(-1).cpu().numpy())[-1]
    return res

In [ ]:
import yaml
res_mcr2 = {}
base_dir = './res_lrd/'
for dim in [2,3,4,5,6,7,8,9,10]:
    latent_dim = dim
    class_dim = dim
    dim_dir = os.path.join(base_dir, 'latent_dim_'+str(latent_dim))
    for ratio in [0]:
        ratio_dir = os.path.join(dim_dir, 'label_ratio_'+str(ratio))
        # 读取数据
        data_dir =os.path.join('./dataloaders/', 'latent_dim_'+str(latent_dim), 'label_ratio_'+str(ratio))
        data_loader, data_set = torch.load(os.path.join(data_dir, 'dataset.pth'), map_location=DEVICE)
        
        # 准备模型
        params_path = os.path.join('model_params', 'mnist_lrd.yaml')
        mnist_params = yaml.load(open(params_path, 'r'), Loader=yaml.FullLoader)

        encoder_params = mnist_params['encoder_params']
        decoder_params = mnist_params['decoder_params']
        encoder_params['class_dim'] = class_dim
        encoder_params['latent_dim'] = latent_dim
        decoder_params['class_dim'] = class_dim
        decoder_params['latent_dim'] = latent_dim
        
        for kl_weights in [0.001]:
            kl_dir = os.path.join(ratio_dir, 'kl_'+str(kl_weights))
            model_dir = os.path.join(kl_dir, 'models')
            #ls = os.listdir(model_dir)
            #ls.sort()
            #best_start = ls[-1][-5]
            best_model_path = os.path.join(model_dir, f'best_overall_model_MCR2.pth')
            print(best_model_path)
            try:
                state_dict = torch.load(best_model_path, weights_only=True, map_location=DEVICE)[0]
            except:
                state_dict = torch.load(best_model_path, weights_only=True, map_location=DEVICE)
            mcr2 =  mcr2_model.MCR2(encoder_params, decoder_params, is_ts=0, kl_weights=kl_weights)
            mcr2.to(DEVICE)
            mcr2.lambdas = 1e-20
            mcr2.load_state_dict(state_dict)
            mcr2.eval()
            output = cal_test_performance(data_set,mcr2, set_name='test')
            #output['time'] = np.array([item['time_consume'] for k,item in res.items()]).mean()
            res_mcr2[kl_weights, ratio, dim] = output

pd.DataFrame(res_mcr2).T

./test_al_0606_mcr2/latent_dim_2\label_ratio_0\kl_0.001\models\best_overall_model_MCR2.pth
./test_al_0606_mcr2/latent_dim_3\label_ratio_0\kl_0.001\models\best_overall_model_MCR2.pth
./test_al_0606_mcr2/latent_dim_4\label_ratio_0\kl_0.001\models\best_overall_model_MCR2.pth
./test_al_0606_mcr2/latent_dim_5\label_ratio_0\kl_0.001\models\best_overall_model_MCR2.pth
./test_al_0606_mcr2/latent_dim_6\label_ratio_0\kl_0.001\models\best_overall_model_MCR2.pth
./test_al_0606_mcr2/latent_dim_7\label_ratio_0\kl_0.001\models\best_overall_model_MCR2.pth
./test_al_0606_mcr2/latent_dim_8\label_ratio_0\kl_0.001\models\best_overall_model_MCR2.pth
./test_al_0606_mcr2/latent_dim_9\label_ratio_0\kl_0.001\models\best_overall_model_MCR2.pth
./test_al_0606_mcr2/latent_dim_10\label_ratio_0\kl_0.001\models\best_overall_model_MCR2.pth


loss  reconstruction_loss    kl_loss       acc
0.001 0 2   0.006343             0.018675 -12.332407  0.990529
        3  -0.009471             0.022551 -32.021820  0.980671
        4  -0.022517             0.022736 -45.253498  0.967488
        5  -0.026279             0.023160 -49.438339  0.978595
        6  -0.027019             0.022557 -49.575577  0.667314
        7  -0.033108             0.017404 -50.512268  0.703386
        8  -0.037976             0.013374 -51.349926  0.781224
        9  -0.040230             0.011726 -51.956116  0.769494
        10 -0.042146             0.010543 -52.689167  0.702000

In [ ]:
import yaml
res_mcr2 = {}
base_dir = './res_lrd/'
for dim in [6,7,8,9,10]:
    latent_dim = dim
    class_dim = dim
    dim_dir = os.path.join(base_dir, 'latent_dim_'+str(latent_dim))
    for ratio in [1/16]:
        ratio_dir = os.path.join(dim_dir, 'label_ratio_'+str(ratio))
        # 读取数据
        data_dir =os.path.join('./dataloaders/', 'latent_dim_'+str(latent_dim), 'label_ratio_'+str(ratio))
        data_loader, data_set = torch.load(os.path.join(data_dir, 'dataset.pth'), map_location=DEVICE)
        
        # 准备模型
        params_path = os.path.join('model_params', 'mnist_lrd.yaml')
        mnist_params = yaml.load(open(params_path, 'r'), Loader=yaml.FullLoader)

        encoder_params = mnist_params['encoder_params']
        decoder_params = mnist_params['decoder_params']
        #kl_weights = mnist_params['kl_weights']
        encoder_params['class_dim'] = class_dim
        encoder_params['latent_dim'] = latent_dim
        decoder_params['class_dim'] = class_dim
        decoder_params['latent_dim'] = latent_dim
        
        for kl_weights in [0.001]:
            kl_dir = os.path.join(ratio_dir, 'kl_'+str(kl_weights))
            model_dir = os.path.join(kl_dir, 'models')
            ls = os.listdir(model_dir)
            ls.sort()
            best_start = ls[-1][-5]
            best_model_path = os.path.join(model_dir, f'best_overall_model_MCR2_{best_start}.pth')
            print(best_model_path)
            try:
                state_dict = torch.load(best_model_path, weights_only=True, map_location=DEVICE)[0]
            except:
                state_dict = torch.load(best_model_path, weights_only=True, map_location=DEVICE)
            mcr2 =  mcr2_model.MCR2(encoder_params, decoder_params, is_ts=0, kl_weights=kl_weights)
            mcr2.to(DEVICE)
            mcr2.lambdas = 1e-20
            mcr2.load_state_dict(state_dict)
            mcr2.eval()
            output = cal_test_performance(data_set,mcr2, set_name='test')
            #output['time'] = np.array([item['time_consume'] for k,item in res.items()]).mean()
            res_mcr2[kl_weights, ratio, dim] = output
            

./test_al_0608_mcr2/latent_dim_6\label_ratio_0.0625\kl_0.001\models\best_overall_model_MCR2_4.pth
./test_al_0608_mcr2/latent_dim_7\label_ratio_0.0625\kl_0.001\models\best_overall_model_MCR2_2.pth
./test_al_0608_mcr2/latent_dim_8\label_ratio_0.0625\kl_0.001\models\best_overall_model_MCR2_1.pth
./test_al_0608_mcr2/latent_dim_9\label_ratio_0.0625\kl_0.001\models\best_overall_model_MCR2_5.pth
./test_al_0608_mcr2/latent_dim_10\label_ratio_0.0625\kl_0.001\models\best_overall_model_MCR2_2.pth


In [ ]:
#
pd.DataFrame(res_mcr2).T

loss  reconstruction_loss    kl_loss       acc
0.001 0.0625 6  -0.033893             0.016049 -49.942066  0.840805
             7  -0.035846             0.013495 -49.341843  0.923820
             8  -0.040239             0.011085 -51.324081  0.914834
             9  -0.041608             0.010431 -52.039158  0.912774
             10 -0.042810             0.009391 -52.201359  0.909833